# Sequence Labeling with Weighted Finite State Machines

- Language Understanding Systems Lab
- Evgeny A. Stepanov
- stepanov.evgeny.a@gmail.com

This notebook is part of the Laboratory Work for [Language Understanding Systems class](http://disi.unitn.it/~riccardi/page7/page13/page13.html) of [University of Trento](https://www.unitn.it/en).
Laboratory has been ported to jupyer notebook format for remote teaching during [COVID-19 pandemic](https://en.wikipedia.org/wiki/2019%E2%80%9320_coronavirus_pandemic).

__Requirements__

- [OpenFST](http://www.openfst.org/twiki/bin/view/FST/WebHome)
- [OpenGRM](http://www.opengrm.org/twiki/bin/view/GRM/NGramLibrary)
- [NL2SparQL4NLU](https://github.com/esrel/NL2SparQL4NLU) dataset

## 1. Shallow Parsing with WFSMs: Natural Language Understanding

Language Understanding of several tasks, one of which is entity extraction (concept tagging). The task is usually approached as Shallow Parsing, where we segment the input into constituents and label them using IOB-schemes.

Using Weighted Finite State Machines for the task provides several benefits:
- Even though we saw how to do sequence labeling using HMM, in real applications models can become quite complex to solve. 
- The task usually involves several components (e.g. *emission* & *transition* probabilities), and WFSMs provide an efficient way to represent and process this components via intersection and composition operations.
    - WFSTs are good at modeling HMM and solving state machine problems
    - Weights can be associated with edges as costs or probabilities (default: cost = negative log probability)

### 1.1. Common Sequence Labeling Pipeline

The common approach to concept tagging (or sequence labeling in general) makes use of 3 components:

|                   | Description                      
|:------------------|:------------------------------
| $$\lambda_{W}$$   | FSA representation of an input sentence
| $$\lambda_{W2T}$$ | FST to translate words into output labels (e.g. `iob+type`)
| $$\lambda_{*LM}$$ | FSA Ngram Language Model to score the sequences of output labels

Consequently, Sequence Labeling ($\lambda$) is performed by composition of these three components as:

$$\lambda = \lambda_{W} \circ \lambda_{W2T} \circ \lambda_{*LM}$$

- It is common to include other components to perform intermediate operations for:
    - generalization of input ($\lambda_{G}$)
    - cleaning of output
    - etc.

### 1.2. General Setup
Let's start by preparing our workspace.

What we have is:
- training & test sets in utterance-per-line format
- training & test sets in CoNLL format that contain word-tag observations

For working with WFSMs we need:
- input symbol table: words
- output symbol table: tags

#### Corpus Preprocessing:
To handle OOV (unknown) words let's:
- apply frequency cut-off to lexicon
- replace OOV words in both training and test data with `<unk>`

##### OOV with Frequency Cut-off using OpenGRM NGram Library Tools

It is easy to apply frequency cut-off and replace OOV with `<unk>` using other means (e.g. `python`). 
Here we demonstrate how to achieve that using provided tools and some `unix` commands.

__Objective__: map low frequency unigrams to `<unk>`

- generate symbol table using `ngramsymbols` for a corpus
- compile the corpus into FAR using `farcompilestrings`
- count unigrams in FAR using `ngramcount`
- print counts using `ngramprint`
- filter the words externally and save them into a file
- generate a new symbol table using `ngramsymbols`
- recompile the corpus into a new FAR using the new symbol table and `farcompilestrings`

> __Note__: *if you provide an external word list to `ngramsymbols` it will generate a symbol table in the required format.*

- Since we will be using corpus files a lot, let's copy them into current directory with shorter names.

In [18]:
%%bash
dpath='NL2SparQL4NLU/dataset/NL2SparQL4NLU'

cp $dpath.train.utterances.txt trn.txt
cp $dpath.test.utterances.txt tst.txt

cp $dpath.train.conll.txt trn.conll
cp $dpath.test.conll.txt tst.conll

- Handling OOV with OpenFST and OpenGRM tools

In [ ]:
%%bash
# creat full symbol table
ngramsymbols trn.txt trn.isyms.tmp
# compile into FAR
farcompilestrings --symbols=trn.isyms.tmp --keep_symbols trn.txt trn.far.tmp
# count unigrams
ngramcount --order=1 trn.far.tmp trn.cnt.tmp
# print counts as integers
ngramprint --integers trn.cnt.tmp trn.cnt.txt.tmp

# bash: you can use python to process file
while read -r word freq; do \
    if (( freq > 1 )); then echo $word ; fi \
done < trn.cnt.txt.tmp > trn.cnt.txt.cutoff.tmp

# final input symbol table
ngramsymbols trn.cnt.txt.cutoff.tmp isyms.txt

# delete temp files
rm *.all

In [23]:
%%bash
# let's compile both training and test set into far using this symbol table
farcompilestrings \
    --symbols=isyms.txt \
    --keep_symbols \
    --unknown_symbol='<unk>' \
    trn.txt trn.far

farcompilestrings \
    --symbols=isyms.txt \
    --keep_symbols \
    --unknown_symbol='<unk>' \
    tst.txt tst.far

As a result we have:
- Symbol table (`isyms.txt`)
    - contains `['<s>', '</s>', '<epsilon>', '<unk>']` that are added automatically
- Training data as FAR with OOV replaced (`trn.far`)
- Test data as FAR with OOV replaced (`tst.far`)

##### Generating Lexicon from CoNLL Format Corpus
To do sequence labeling we additionally require *output symbol table*.
We can generate it using simple `unix` commands and `ngramsymbols` from CoNLL format corpora.
- create a unique list of output symbols using `unix`
- generate symbol table using `ngramsymbols`

Even though our test set might contain __types__ not present in the training set, since they are required only for evaluation, our symbol table can ignore them. 

Since our output labels are composed of segmentation(`iob`) and classification (`type`) labels, we can make sure that each __type__ has all possible IOB prefixes (even those not present in training data).

In [24]:
%%bash
# create a unique tag list
cat trn.conll | cut -f 2 | cut -d '-' -f 2 | sed '/^ *$/d' | sort | uniq > types.txt

while read -r word
do
    if [[ $word != 'O' ]]
    then
        echo "B-$word"
        echo "I-$word"
    else
        echo $word
    fi
done < types.txt > osyms.tmp

# generate symbol table with tags
ngramsymbols osyms.tmp osyms.txt

rm *.tmp

### 1.3. Applying FSTs
- Our objective is to be able to annotate input sentences (as FSAs in FAR) using the machines we are going to build. 
- Our test data has been loaded into FAR archive
- We will can `farextract` to extract these sentence FSAs and apply our FSMs to them
    - `farextract --filename_prefix="<odir>" <FAR>` will extract contents of `<FAR>` into directory `<odir>`
    - we can iterate over files in the directory and apply operations (see below)
- We can also create FAR of the processed FSMs using `farcreate` as
    - `farcreate --file_list_input <list of FST filenames> <output FAR>`


In [25]:
%%bash
wdir='wdir'
mkdir -p $wdir

farextract --filename_prefix="$wdir/" tst.far
cp $wdir/tst.txt-0001 sent.fsa

fstprint sent.fsa

0	1	star	star
1	2	of	of
2	3	<unk>	<unk>
3


- For testing we are going to use this `sent.fsa`
- For evaluation we are going to iterate over whole FAR

### 1.3. Baselines
Let's demonstrate the process by building some simple Shallow Parsing models.

### 1.3.1. Random
The simplest solution is to assign output labels randomly. 

To achieve this we need to implement an FST that translates our words into output symbols ($\lambda_{W2T}$) with equal cost, or no cost at all (i.e. unweighted FST -- let's call it $\lambda_{U}$ for "unweighted"). 

The resulting FST represents search space for $p(w_i|t_i)$ without being exposed to any observation. It is build using only our knowledge of input and output symbols and all translations are possible. 

Since we have no model yet, the whole pipeline is:

$$\lambda = \lambda_{W} \circ \lambda_{W2T_{U}}$$

In [26]:
%%bash
# Let's read our symbol tables into arrays, removing special symbols except '<unk>' for input
# and removing all of them for output (to avoid <unk> - <unk>, which we can still keep)
ist_arr=($(cat isyms.txt | sed '/<epsilon>/d;/<s>/d;/<\/s>/d' | cut -f 1 ))
ost_arr=($(cat osyms.txt | sed '/<epsilon>/d;/<s>/d;/<\/s>/d;/<unk>/d' | cut -f 1 ))

# creating transducer
for is in ${ist_arr[@]}
do
    for os in ${ost_arr[@]}
    do
        echo "0 0 $is $os"
    done
done > w2t.u.txt
# add final state
echo "0" >> w2t.u.txt

# Let's compile it
fstcompile \
    --isymbols=isyms.txt \
    --osymbols=osyms.txt \
    --keep_isymbols \
    --keep_osymbols \
    w2t.u.txt w2t.u.bin

fstinfo w2t.u.bin | head -n 8

fst type                                          vector
arc type                                          standard
input symbol table                                isyms.txt
output symbol table                               osyms.txt
# of states                                       1
# of arcs                                         44697
initial state                                     0
# of final states                                 1


#### Exercise
- Compute input and output symbol table sizes
- Compare their multiplication to `# of arcs`

#### Testing

In [27]:
%%bash
fstcompose sent.fsa w2t.u.bin | fstshortestpath | fstrmepsilon | fsttopsort | fstprint

0	1	star	O
1	2	of	O
2	3	<unk>	O
3


#### Evaluation

- For evaluation we are going to use `conlleval.pl` script (provided is `src` directory)
- We need to convert output to the appropriate format
- All tokens will be predicted as `O`.
    - $F_1=0$
    - Bonus Question: *Why?*

In [28]:
%%bash
wdir='wdir'
farr=($(ls $wdir))

for f in ${farr[@]}
do
    fstcompose $wdir/$f w2t.u.bin | fstshortestpath | fstrmepsilon | fsttopsort | fstprint
done > w2t.u.out

In [29]:
%%bash
paste tst.conll w2t.u.out | cut  -f 1,2,6 | tr '\t' ' ' | sed 's/^ .*//g' > w2t.u.out.conll 
perl ../src/conlleval.pl < w2t.u.out.conll                                                             

processed 7117 tokens with 1091 phrases; found: 0 phrases; correct: 0.
accuracy:  72.15%; precision:   0.00%; recall:   0.00%; FB1:   0.00
       actor.name: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
actor.nationality: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
       actor.type: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
   award.category: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
   award.ceremony: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
   character.name: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
     country.name: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
    director.name: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
director.nationality: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
      movie.genre: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
movie.gross_revenue: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
   movie.language: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
   m

### 1.3.2. Output Symbol Priors

The simplest form for the third component is to use output label priors, i.e. unigram probabilities of output labels. To model that we can:
- train a unigram language model using `ngramcount` & `ngrammake` (let's call it $\lambda_{LM_{1}}$)
- compose it with the *random* $\lambda_{W2T_{U}}$ so that the whole pipeline becomes:

$$\lambda = \lambda_{W} \circ \lambda_{W2T_{U}} \circ \lambda_{LM_{1}}$$

- In this baseline our model becomes exposed to observations
- Since `O` tag is the most frequent & we will have a model that always predicts it.
- We can represent our model as:

$$p(t_{1}^{n}|w_{1}^{n}) \approx \prod_{i=1}^{n}{p(t_i)}$$

Let's create corpus of output labels in sentence-per-line format & create FAR.

In [30]:
%%bash
cat trn.conll | cut -f 2 |\
    sed 's/^$/~/g' | tr '\n' ' ' | tr '~' '\n' |\
    sed 's/  */ /g;s/^ *//g;s/ *$//g' > trn.t.txt

farcompilestrings \
    --symbols=osyms.txt \
    --keep_symbols \
    --unknown_symbol='<unk>' \
    trn.t.txt trn.t.far

- Let's train a unigram language model and evaluate it.

In [31]:
%%bash
ngramcount --order=1 trn.t.far trn.t.1.cnt
ngrammake trn.t.1.cnt t.1.lm
ngraminfo t.1.lm

# of states                                       1
# of ngram arcs                                   41
# of backoff arcs                                 0
initial state                                     0
unigram state                                     -1
# of final states                                 1
ngram order                                       1
# of 1-grams                                      42
well-formed                                       y
normalized                                        y


In [38]:
%%bash
fstcompose sent.fsa w2t.u.bin | fstcompose - t.1.lm | fstshortestpath | fstrmepsilon | fsttopsort | fstprint

0	1	star	O	0.476737976
1	2	of	O	0.476737976
2	3	<unk>	O	0.476737976
3	2.00485039


In [41]:
%%bash
wdir='wdir'
farr=($(ls $wdir))

for f in ${farr[@]}
do
    fstcompose $wdir/$f w2t.u.bin | fstcompose - t.1.lm |\
        fstshortestpath | fstrmepsilon | fsttopsort | fstprint
done > w2t.u.t.1.out

paste tst.conll w2t.u.t.1.out | cut  -f 1,2,6 | tr '\t' ' ' | sed 's/^ .*//g' > w2t.u.t.1.out.conll
perl ../src/conlleval.pl < w2t.u.t.1.out.conll

processed 7117 tokens with 1091 phrases; found: 0 phrases; correct: 0.
accuracy:  72.15%; precision:   0.00%; recall:   0.00%; FB1:   0.00
       actor.name: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
actor.nationality: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
       actor.type: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
   award.category: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
   award.ceremony: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
   character.name: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
     country.name: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
    director.name: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
director.nationality: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
      movie.genre: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
movie.gross_revenue: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
   movie.language: precision:   0.00%; recall:   0.00%; FB1:   0.00  0
   m

- The model still has $F_1=0$, since `O` is the tag with highest prior.

### 1.3.4. Bigram Model: Exercise
- Train a *tag* bigram model (let's call it $\lambda_{LM_{2}}$)
- Test pipeline with $\lambda_{W2T_{U}}$ as 

$$\lambda = \lambda_{W} \circ \lambda_{W2T_{U}} \circ \lambda_{LM_{2}}$$

- Evaluate it

### 1.3.5. Maximum Likelihood Estimation (Emission Probabilities)
- So far we haven't explored the relation between input and output
- The next thing we can do is to expose our model to observations and estimate $p(w_{i}|t_{i})$ from data.
- We can use `ngramcount` and `ngrammake` to make a smoothed probability model (we are using default, i.e. no parameters). 
- We need to estimate probabilities like we would estimate bigram probabilities, thus:
    - prepare lexicon with *tags* and *words*
    - read CoNLL format corpus into far (token per line, preprocessed)
    - count bigrams
    - make a bigram language model
    - print bigrams with weights (negative log probabilities)
    - choose bigrams (it will contain unigrams, as well as `<s>` and `</s>` bigrams)
    - convert to FST & compile
    
- Let's call them model $\lambda_{W2T_{MLE}}$

In [36]:
%%bash
# lets use our symbol tables
cat isyms.txt osyms.txt | cut -f 1 | sort | uniq > msyms.text.tmp
ngramsymbols msyms.text.tmp msyms.txt

# let's convert data to ngrams
cat trn.conll | sed '/^$/d' | awk '{print $2,$1}' > trn.w2t.txt

# compile to far
farcompilestrings \
    --symbols=msyms.txt \
    --keep_symbols \
    --unknown_symbol='<unk>' \
    trn.w2t.txt trn.w2t.far
    
# count bigrams
ngramcount --order=2 trn.w2t.far trn.w2t.cnt
# make a model
ngrammake trn.w2t.cnt trn.w2t.lm

# print ngram probabilities as negative logs
ngramprint \
    --symbols=msyms.txt\
    --negativelogs \
    trn.w2t.lm trn.w2t.probs

# since the model also contains unigrams and initial and final symbols ngrams, 
# let's get ngrams starting with iob-prefix
cat trn.w2t.probs | grep '^[IB]-\|O ' | tr '\t' ' ' | sed 's/^/0 0 /g' > tmp.tmp

# remove remaining unigrams
while read line
do
    len=$(echo "$line" | grep -o " " | wc -l)
    # 4 space == 5 columns
    if (( len == 4 ))
    then
        echo "$line"
    fi
done < tmp.tmp > trn.w2t.mle.txt

# adding final state
echo '0' >> trn.w2t.mle.txt

fstcompile \
    --isymbols=osyms.txt \
    --osymbols=isyms.txt \
    --keep_isymbols \
    --keep_osymbols \
    trn.w2t.mle.txt w2t.mle.bin
    
# we need to invert it to have words on input
fstinvert w2t.mle.bin w2t.mle.inv.bin

fstinfo w2t.mle.inv.bin | head -n 8

fst type                                          vector
arc type                                          standard
input symbol table                                isyms.txt
output symbol table                               osyms.txt
# of states                                       1
# of arcs                                         1513
initial state                                     0
# of final states                                 1


#### Testing
Let's test it:

In [37]:
%%bash
fstcompose sent.fsa w2t.mle.inv.bin | fstshortestpath | fstrmepsilon | fsttopsort | fstprint

0	1	star	B-movie.name	3.22130394
1	2	of	I-movie.name	3.02857399
2	3	<unk>	B-director.nationality	0.694147706
3


- The pipeline above represents 

$$p(t_{1}^{n}|w_{1}^{n}) \approx \prod_{i=1}^{n}{p(w_i|t_i)}$$

- To extend it to unigram tagging model we need to compose it with  $\lambda_{LM_{1}} = p(t_i)$ 

$$\lambda = \lambda_{W} \circ \lambda_{W2T_{MLE}} \circ \lambda_{LM_{1}}$$

$$p(t_{1}^{n}|w_{1}^{n}) \approx \prod_{i=1}^{n}{p(w_i|t_i)p(t_i)}$$ 

In [39]:
%%bash
fstcompose sent.fsa w2t.mle.inv.bin | fstcompose - t.1.lm | fstshortestpath | fstrmepsilon | fsttopsort | fstprint

0	1	star	B-movie.name	6.09392548
1	2	of	O	3.86930203
2	3	<unk>	O	4.46328497
3	2.00485039


#### Exercise 1: Maximum Likelihood Estimation
- using `ngramprint` verify the Maximum Likelihood Estimation method
    - print bigram counts for $\lambda_{W2T_{MLE}}$ (output of `ngramcount`)
    - print unigram counts for $\lambda_{LM_{1}}$ (output of `ngramcount`)
    - compute negative log probabilities using those counts
    - compare values to the ones extracted from the $\lambda_{W2T_{MLE}}$ (output of `ngrammake`)

#### Exercise 2: HMM Tagger
- Evaluate the MLE pipeline using bigram model on tags, i.e.

$$\lambda = \lambda_{W} \circ \lambda_{W2T_{MLE}} \circ \lambda_{LM_{2}}$$

$$p(t_{1}^{n}|w_{1}^{n}) \approx \prod_{i=1}^{n}{p(w_i|t_i)p(t_i|t_{i-1})}$$ 

- compare performances to the HMM tagger from previous lab (NLTK)

## 2. Joint Distribution Modeling

As we have seen, sequence labeling for Language Understanding could be approached using Hidden Markov Models (similar to Part-of-Speech Tagging), and to models it as in the table below (__HMM__). Stochastic Conceptual Language Models for Spoken Language Understanding in [Raymond & Riccardi (2007)](https://disi.unitn.it/~riccardi/papers2/IS07-GenerDiscrSLU.pdf) (__R&R__) model it jointly.


| Model   | Equation |
|:--------|:----------
| __HMM__ | $$p(t_{1}^{n}|w_{1}^{n}) \approx \prod_{i=1}^{n}{p(w_i|t_i)p(t_i|t_{i-N+1}^{i-1})}$$
| __R&R__ | $$p(w_{1}^n,t_{1}^{n}) \approx \prod_{i=1}^{n}{p(w_{i}t_{i}|w_{i-N+1}^{i-1}t_{i-N+1}^{i-1})}$$


From implementation perspective, joint modeling implies the following:
- we need to train $\lambda_{SCLM}$ on word-tag pairs
    - create corpus in a format for estimating $p(w_i,t_i|w_{i-N+1}^{i-1}t_{i-N+1}^{i-1})$
    - create symbol tables
- we need to change $\lambda_{W2T}$ to output *word-tag* pairs (let's call it $\lambda_{W2WT}$)
    - create FST like above for $\lambda_{W2WT}$

#### Preparing Symbol Tables
- Let's create symbol tables the same way we did for $\lambda_{W2T_{U}}$

In [55]:
%%bash
# Let's read our symbol tables into arrays, removing special symbols except '<unk>' for input
# and removing all of them for output (to avoid <unk> - <unk>, which we can still keep)
ist_arr=($(cat isyms.txt | sed '/<epsilon>/d;/<s>/d;/<\/s>/d' | cut -f 1 ))
ost_arr=($(cat osyms.txt | sed '/<epsilon>/d;/<s>/d;/<\/s>/d;/<unk>/d' | cut -f 1 ))

# creating transducer
for is in ${ist_arr[@]}
do
    for os in ${ost_arr[@]}
    do
        echo "0 0 $is ${is}+${os}"
    done
done > w2wt.u.txt
# add final state
echo "0" >> w2wt.u.txt

# creating input for output symbol tables
for is in ${ist_arr[@]}
do
    for os in ${ost_arr[@]}
    do
        echo "${is}+${os}"
    done
done > wt.osyms.tmp

ngramsymbols wt.osyms.tmp wt.osyms.txt
rm *.tmp

# Let's compile it
fstcompile \
    --isymbols=isyms.txt \
    --osymbols=wt.osyms.txt \
    --keep_isymbols \
    --keep_osymbols \
    w2wt.u.txt w2wt.u.bin

fstinfo w2wt.u.bin | head -n 8

fst type                                          vector
arc type                                          standard
input symbol table                                isyms.txt
output symbol table                               wt.osyms.txt
# of states                                       1
# of arcs                                         44697
initial state                                     0
# of final states                                 1


In [49]:
%%bash
# using `~` for sentence separation since it is not in data
# using `+` to represet W+T (like in symbol table)

cat trn.conll | tr '\t' '+' |\
    sed 's/^$/~/g' | tr '\n' ' ' | tr '~' '\n' |\
    sed 's/  */ /g;s/^ *//g;s/ *$//g' > trn.wt.txt

farcompilestrings \
    --symbols=wt.osyms.txt \
    --keep_symbols \
    --unknown_symbol='<unk>' \
    trn.wt.txt trn.wt.far

- Let's train an ngram model on it

In [51]:
%%bash
ngramcount --order=2 trn.wt.far trn.wt.cnt
ngrammake trn.wt.cnt wt.2.lm
ngraminfo wt.2.lm

# of states                                       1484
# of ngram arcs                                   7073
# of backoff arcs                                 1483
initial state                                     1
unigram state                                     0
# of final states                                 638
ngram order                                       2
# of 1-grams                                      1483
# of 2-grams                                      6228
well-formed                                       y
normalized                                        y


- Lets test the whole $\lambda_{W} \circ \lambda_{W2WT} \circ \lambda_{SCLM_{2}}$

In [54]:
%%bash
fstcompose sent.fsa w2wt.u.bin | fstcompose - wt.2.lm | fstshortestpath | fstrmepsilon | fsttopsort | fstprint